# Variational quantum eigensolving

In this exercise, we exemplify how the VQE method can be used to find the approximate ground state (GS) of a given Hamiltonian. Here, we take a random Hamiltonian for concreteness.

## Defining the Hamiltonian

Here, we define a random Hamiltonian with 5 terms acting on 3 qubits. In myQLM, Hamiltonians are implemented with ``Observable`` objects (https://myqlm.github.io/qat-core.html#module-qat.core.wrappers.observable).

In [ ]:
import numpy as np
from qat.core import Observable, Term

def make_random_hamiltonian(nqbits, nterms):
    terms = []
    for _ in range(nterms):
        coeff = np.random.random()
        ops = "".join(np.random.choice(["X", "Z"], size=nqbits))
        qbits = np.random.choice(nqbits, size=nqbits, replace=False)
        terms.append(Term(coefficient=coeff, pauli_op=ops, qbits=qbits))
    hamiltonian = Observable(nqbits, pauli_terms=terms)
    return hamiltonian


nqbits = 3
nterms = 5
np.random.seed(1423543) #fixing seed to have reproducible results
hamiltonian = make_random_hamiltonian(nqbits, nterms)
print("H:", hamiltonian)

Because $H$ is defined on a small number of qubits, we can afford to diagonalize it exactly and thus compute the exact GS energy:

In [ ]:
from utils_tuto import make_matrix
H_mat = make_matrix(hamiltonian)
eigvals = np.linalg.eigvalsh(H_mat)
E0 = min(eigvals)

What is the size of the matrix representation of $H$? For a number of qubits $n$, what is the size in general?

## Constructing a variational circuit

The main task in VQE consists in designing a 'good' ansatz, i.e a family of variational states that can come close enough to the exact ground state. These states are constructed using quantum circuits with parameters (like rotation angles) that will be optimized to minimize the energy.

In [ ]:
from qat.lang.AQASM import Program, QRoutine, RY, CNOT, RX, Z, H, RZ

def make_ansatz():
    """Function that prepares an ansatz circuit
    
    Returns:
        Circuit: a parameterized circuit (i.e with some variables that are not set)
    """
    
    nparams = ... # number of parameters 
    
    prog = Program()
    reg = prog.qalloc(nqbits)
    # define variables using 'new_var' 
    theta = [prog.new_var(float, '\\theta_%s'%i)
             for i in range(nparams)]
    
    # for instance...: put a rotation on each qubit
    for ind in range(nqbits):
        RY(theta[ind])(reg[ind])
        
    # and so on
    # ...
    
    circ = prog.to_circ()
    return circ

circ = make_ansatz()
%qatdisplay circ


## Creating a variational job and a variational stack

We now create a variational job from this circuit and observable.
It is then submitted to a "variational stack" composed of a perfect QPU, ``LinAlg``, and a variational plugin, ``ScipyMinimizePlugin``. The latter handles parametric jobs. These are jobs whose circuit contains a parameter that is then going to be optimized, using classical optimizers, so as to minimize the value of the observable over the final state.

Below, we are going to test three different classical optimizers: COBYLA, Nelder-Mead, and BFGS.

In [ ]:
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin #,SPSAMinimizePlugin

circ = make_ansatz()

job = circ.to_job(job_type="OBS",
                  observable=hamiltonian,
                  nbshots=100)
# note that this job contains a circuit whose variables are not set!
# it therefore requires a special plugin so that the QPU can handle it!

theta_0 = ... # initial value of the parameters

linalg_qpu = get_default_qpu()
method = "COBYLA"  # look at the doc of scipy.optimize.minimize for other options
optimizer_scipy = ScipyMinimizePlugin(method=method,
                                      tol=1e-6,
                                      options={"maxiter": 200},
                                      x0=theta_0)
# you could also use SPSAMinimizePlugin instead

# we now build a stack that can handle variational jobs
qpu = optimizer_scipy | linalg_qpu

# we submit the job
result = qpu.submit(job)

print("Minimum VQE energy (%s) = %s"%(method, result.value))

What do you observe? It the VQE energy close to the exact energy ``E0``? If not, where does it come from: a poor ansatz? a poor optimizer? a poor QPU? not enough ``nbshots``? Try playing with these various knobs. Hint: entanglement helps!


## Plotting the results

We can also plot the value of the variational energy over the course of the classical optimization. For this, we can retrieve information about the variational job execution in the ``meta_data`` field of the result.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(eval(result.meta_data['optimization_trace']),
         label=method)
    
plt.plot([E0 for _ in range(300)], '--k', lw=3, label="exact energy")
    
plt.grid()
plt.legend(loc="best")
plt.xlabel("Steps")
plt.ylabel("Energy");

What can you observe in terms of convergence speed?

Once you have found an ansatz and an optimizer that works with a perfect QPU, can you now test what happens with a noisy QPU using the ``DepolarizingPlugin`` we already used in previous notebooks?